In [3]:
from urllib.request import urlopen
import urllib
from urllib.request import Request
from bs4 import BeautifulSoup
import pandas as pd
from user_agent import generate_user_agent

url_1 = "https://www.imdb.com/title/tt0898266/episodes/?season={num}&ref_=ttep"
url_2 = "https://www.imdb.com"
epi_list = []

for i in range(1, 13):
    try:
        req = Request(url=url_1.format(num=i), headers={'User-Agent':generate_user_agent()})
        page = urlopen(req)
        soup = BeautifulSoup(page, "html.parser")
        
        episodes = soup.find(class_="sc-67c7a421-0 jsgvdx")
        if episodes is None:
            print(f"Season {i}: No episodes found")
            continue
            
        for idx, each in enumerate(episodes):
            try:
                epi = []
                
                # 제목과 에피소드 정보 가져오기
                title_element = each.find(class_="ipc-title__text")
                if title_element:
                    full_title = title_element.get_text().split(" ∙ ")
                    sea_epi = full_title[0].split(".")
                    season = int(sea_epi[0].replace('S', ''))
                    episode = int(sea_epi[1].replace('E', ''))
                    title = full_title[1]
                else:
                    continue
                
                # 날짜 정보 가져오기
                date_element = each.find(class_="sc-f2169d65-10 bYaARM")
                date = date_element.get_text() if date_element else "No date"
                
                # 링크 정보 가져오기
                link_element = each.find(class_="ipc-lockup-overlay ipc-focusable")
                if link_element and 'href' in link_element.attrs:
                    link = link_element['href']
                    url_link = url_2 + link
                    
                    # 상세 페이지 접근
                    req_link = Request(url=url_link, headers={'User-Agent':generate_user_agent()})
                    page_link = urlopen(req_link)
                    soup_link = BeautifulSoup(page_link, "html.parser")
                    
                    # 장르 및 감독, 작가, 배우 정보 가져오기
                    genre_staff_element = soup_link.find(class_="sc-70a366cc-4 jrGDsj")
                    if genre_staff_element:
                        genre_tags = genre_staff_element.find_all("span", "ipc-chip__text")
                        genre = ", ".join([tag.get_text() for tag in genre_tags]) if genre_tags else "No genre"

                        staff_tags = genre_staff_element.find_all(
                            "a", "ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link")
                        staff = [tag.get_text() for tag in staff_tags] if staff_tags else "No staff"

                        director = staff[0]
                        writers = staff[1:4]
                        stars = staff[4:]
                    else:
                        genre = "No genre"
                        staff = "No staff"
                    
                    # 내용 정보 가져오기
                    content_element = soup_link.find(class_="ipc-html-content-inner-div")
                    content = content_element.get_text() if content_element else "No content"
                else:
                    continue
                
                # 데이터 추가
                epi.extend([season, episode, title, date, genre, director, writers, stars, content])
                epi_list.append(epi)
                
            except Exception as e:
                print(f"Error processing episode {idx+1} in season {i}: {str(e)}")
                continue
                
    except Exception as e:
        print(f"Error processing season {i}: {str(e)}")
        continue

epi_df = pd.DataFrame(epi_list, 
                      columns=["season", "episode", "title", "date", "genre", "director", "writers", "stars", "content"])
epi_df.head()

,season,episode,title,date,genre,director,writers,stars,content
0,1,1,Pilot,"Mon, Sep 24, 2007","Comedy, Romance",James Burrows,"[Chuck Lorre, Bill Prady, Johnny Galecki]","[Jim Parsons, Kaley Cuoco]",Leonard (Johnny Galecki) states that the combi...
1,1,2,The Big Bran Hypothesis,"Mon, Oct 1, 2007","Comedy, Romance",Mark Cendrowski,"[Chuck Lorre, Bill Prady, Robert Cohen]","[Johnny Galecki, Jim Parsons, Kaley Cuoco]",This is the only time where a different camera...
2,1,3,The Fuzzy Boots Corollary,"Mon, Oct 8, 2007","Comedy, Romance",Mark Cendrowski,"[Chuck Lorre, Bill Prady, Steven Molaro]","[Johnny Galecki, Jim Parsons, Kaley Cuoco]",The song that Leonard (Johnny Galecki) is sing...
3,1,4,The Luminous Fish Effect,"Mon, Oct 15, 2007","Comedy, Romance",Mark Cendrowski,"[Chuck Lorre, Bill Prady, David Litt]","[Johnny Galecki, Jim Parsons, Kaley Cuoco]",When Mary Cooper (Laurie Metcalf) introduces h...
4,1,5,The Hamburger Postulate,"Mon, Oct 22, 2007","Comedy, Romance",Andrew D. Weyman,"[Chuck Lorre, Bill Prady, David Goetsch]","[Johnny Galecki, Jim Parsons, Kaley Cuoco]",Johnny Galecki (Leonard) knows how to play the...


In [10]:
# 리스트를 문자열로 변환하는 함수
def clean_list_to_string(x):
    if isinstance(x, list):
        return ", ".join(x)  # 리스트의 요소들을 쉼표로 구분하여 문자열로 변환
    return x

# writers와 stars 컬럼에 함수 적용
epi_df['writers'] = epi_df['writers'].apply(clean_list_to_string)
epi_df['stars'] = epi_df['stars'].apply(clean_list_to_string)

# 인덱스 설정
#epi_df.set_index(["season", "episode"], inplace=True)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

display(epi_df)

title               date  \
season episode                                                                 
1      1                                            Pilot  Mon, Sep 24, 2007   
       2                          The Big Bran Hypothesis   Mon, Oct 1, 2007   
       3                        The Fuzzy Boots Corollary   Mon, Oct 8, 2007   
       4                         The Luminous Fish Effect  Mon, Oct 15, 2007   
       5                          The Hamburger Postulate  Mon, Oct 22, 2007   
       6                        The Middle Earth Paradigm  Mon, Oct 29, 2007   
       7                             The Dumpling Paradox   Mon, Nov 5, 2007   
       8                       The Grasshopper Experiment  Mon, Nov 12, 2007   
       9               The Cooper-Hofstadter Polarization  Mon, Mar 17, 2008   
       10                            The Loobenfeld Decay  Mon, Mar 24, 2008   
       11                      The Pancake Batter Anomaly  Mon, Mar 31, 2008   
       12                           The Jerusalem Duality  Mon, Apr 14, 2008   
       13                          The Bat Jar Conjecture  Mon, Apr 21, 2008   
       14                       The Nerdvana Annihilation  Mon, Apr 28, 2008   
       15                     The Pork Chop Indeterminacy   Mon, May 5, 2008   
       16                             The Peanut Reaction  Mon, May 12, 2008   
       17                            The Tangerine Factor  Mon, May 19, 2008   
2      1                            The Bad Fish Paradigm  Mon, Sep 22, 2008   
       2                            The Codpiece Topology  Mon, Sep 29, 2008   
       3                        The Barbarian Sublimation   Mon, Oct 6, 2008   
       4                          The Griffin Equivalency  Mon, Oct 13, 2008   
       5                           The Euclid Alternative  Mon, Oct 20, 2008   
       6                      The Cooper-Nowitzki Theorem   Mon, Nov 3, 2008   
       7                    The Panty Piñata Polarization  Mon, Nov 10, 2008   
       8                       The Lizard-Spock Expansion  Mon, Nov 17, 2008   
       9                The White Asparagus Triangulation  Mon, Nov 24, 2008   
       10                       The Vartabedian Conundrum   Mon, Dec 8, 2008   
       11                   The Bath Item Gift Hypothesis  Mon, Dec 15, 2008   
       12                    The Killer Robot Instability  Mon, Jan 12, 2009   
       13                        The Friendship Algorithm  Mon, Jan 19, 2009   
       14                      The Financial Permeability   Mon, Feb 2, 2009   
       15                        The Maternal Capacitance   Mon, Feb 9, 2009   
       16                          The Cushion Saturation   Mon, Mar 2, 2009   
       17                       The Terminator Decoupling   Mon, Mar 9, 2009   
       18                       The Work Song Nanocluster  Mon, Mar 16, 2009   
       19                   The Dead Hooker Juxtaposition  Mon, Mar 30, 2009   
       20                          The Hofstadter Isotope  Mon, Apr 13, 2009   
       21                       The Vegas Renormalization  Mon, Apr 27, 2009   
       22             The Classified Materials Turbulence   Mon, May 4, 2009   
       23                        The Monopolar Expedition  Mon, May 11, 2009   
3      1              The Electric Can Opener Fluctuation  Mon, Sep 21, 2009   
       2                            The Jiminy Conjecture  Mon, Sep 28, 2009   
       3                          The Gothowitz Deviation   Mon, Oct 5, 2009   
       4                              The Pirate Solution  Mon, Oct 12, 2009   
       5               The Creepy Candy Coating Corollary  Mon, Oct 19, 2009   
       6                            The Cornhusker Vortex   Mon, Nov 2, 2009   
       7                      The Guitarist Amplification   Mon, Nov 9, 2009   
       8                     The Adhesive Duck Deficiency  Mon, Nov 16, 2009   
       9                        The Vengeance Formul

In [11]:
epi_df.to_csv("../data/Big_Bang_Theory.csv")